##### Resources setup in AWS
1.  Create your code bucket => s3://codebucketpsspark
2.  Source data bucket => s3://etlbucket/SampleSuperstore.csv
3.  Destination bucket => s3://boto3practticebucket/aggregatedData.csv
4.  Create a role with the following policies : cloudwatch,s3,emr
5.  Create Lambda function and attach role
6.  Setup trigger to the destination s3
6.  Attach lambda function to source data bucket

In [ ]:
import sys
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.types import (StructField,StructType,
                               ShortType,IntegerType,StringType,FloatType,BooleanType)

spark=SparkSession\
         .builder\
         .appName('S3 read and write job')\
         .getOrCreate()


def main():
    s3_bucket = sys.argv[1]
    s3_filename = sys.argv[2]
    s3_location = "s3://{}/{}".format(s3_bucket,s3_filename )
    # Format 1
    iris = spark.read \
            .format("csv") \
            .option("inferSchema","true")\
            .option("header","true") \
            .load(f'{s3_location}')
    
     # Format 2
    superData = spark.read \
                .csv('s3://etlbucket/SampleSuperstore.csv',
                      "header"="true",
                      "inferSchema"="true")
    superData=iris.groupby("region").count()

    # Output Fotmat
    superData.write\
     .format("csv") \
     .option("header","true") \
     .save("s3://etlbucket/aggregatedData.csv")
    
    
if __name__=="__main":
    main()